In [1]:
from pathlib import Path
from omop_alchemy.cdm.base import bootstrap
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

from omop_alchemy import configure_logging, get_engine_name, load_environment, TEST_PATH, ROOT_PATH

from omop_alchemy.model.vocabulary import (
    Domain,
    Vocabulary,
    Concept_Class,
    Relationship,
    Concept,
    Concept_Ancestor,
    Concept_Relationship,
)

ATHENA_LOAD_ORDER = [
    Domain,
    Vocabulary,
    Concept_Class,
    Relationship,
    Concept,
    Concept_Ancestor,
    Concept_Relationship,
]

configure_logging()
load_environment()

engine_string = get_engine_name()
engine = sa.create_engine(engine_string, future=True, echo=False)
bootstrap(engine, create=True)

2025-12-31 15:16:56,791 | INFO     | omop_alchemy.omop_alchemy.config | Environment variables loaded from .env file
2025-12-31 15:16:56,792 | INFO     | omop_alchemy.omop_alchemy.config | Database engine configured
2025-12-31 15:16:56,800 | INFO     | omop_alchemy.omop_alchemy.cdm.base.declarative | Bootstrapping OMOP schema (create=True)
2025-12-31 15:16:56,800 | INFO     | omop_alchemy.omop_alchemy.cdm.base.declarative | Schema creation enabled


In [2]:
Session = sessionmaker(bind=engine, future=True)
session = Session()

In [3]:
base_path = TEST_PATH / "fixtures" / "athena_source"

In [4]:
for model in ATHENA_LOAD_ORDER:
    model.load_csv(session, base_path / f"{model.__tablename__.upper()}.csv")

In [5]:
from sqlalchemy.exc import IntegrityError
from omop_alchemy.cdm.base.declarative import explain_sqlite_fk_error

try:
    session.commit()
except IntegrityError as e:
    print(e)
    explain_sqlite_fk_error(session, e, raise_error=False)
    session.rollback()